<img SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

# Combining and splitting model layers

*D.A. Brakenhoff, Artesia, 2021*

This notebook shows methods for combining layers and splitting layers for MODFLOW models. Multiple layers can be combined into one layer or one layer can be split into sub-layers based on a fraction of the original thickness.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString

import nlmod
from nlmod.plot import DatasetCrossSection

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

In [ ]:
def compare_layer_models(
    ds1,
    line,
    colors,
    ds2=None,
    zmin=-200,
    zmax=10,
    min_label_area=1000,
    title1="REGIS original",
    title2="Modified layers",
    xlabel="Distance along x-sec (m)",
    ylabel="m NAP",
):
    if ds2 is None:
        fig, ax1 = plt.subplots(1, 1, figsize=(14, 6))
    else:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
    dcs1 = DatasetCrossSection(ds1, line=line, ax=ax1, zmin=zmin, zmax=zmax)
    polys2 = dcs1.plot_layers(colors=colors, min_label_area=min_label_area)
    dcs1.plot_grid(linewidth=0.5, vertical=False)
    ax1.set_ylabel(ylabel)

    if ds2 is not None:
        ax1.set_title(title1)
        dcs2 = DatasetCrossSection(ds2, line=line, ax=ax2, zmin=zmin, zmax=zmax)
        polys1 = dcs2.plot_layers(colors=colors, min_label_area=min_label_area)
        dcs2.plot_grid(linewidth=0.5, vertical=False)
        ax2.set_ylabel(ylabel)
        ax2.set_xlabel(xlabel)
        ax2.set_title(title2)
    else:
        ax1.set_xlabel(xlabel)

## Get data

Define an extent to obtain REGIS

In [ ]:
extent = [131000, 136800, 471500, 475700]

Download and cache REGIS netCDF.

In [ ]:
regis = nlmod.read.regis.download_regis(extent)

Let's take a look at the dataset

In [ ]:
regis

Define an line to draw a cross-section

In [ ]:
# diagonal line through extent
line = LineString([(extent[0], extent[2]), (extent[1], extent[3])])

Get colors for our cross-section plot.

In [ ]:
colors = nlmod.read.regis.get_legend()["color"].to_dict()

Draw the cross-section for REGIS

In [ ]:
compare_layer_models(regis, line, colors)

## Split layers

First we determine how to split the layers. This is done by creating a list of factors,
that is used to determine fractions that add up to 1. The layer will be split into
sub-layers from the top down, with each sub-layer getting a thickness equal to the
fraction times the original thickness.

For example, `(1, 1)` will split the layer into two sub-layers, each getting a
thickness equal to 50% of the original layer. In this example the fractions already add
up to 1 for each layer.

In [ ]:
# split dictionary
split_dict = {
    "PZWAz2": (0.3, 0.3, 0.4),
    "PZWAz3": (0.2, 0.2, 0.2, 0.2, 0.2),
}

Calculate the new layer elevations based on the information above.

In [ ]:
regis_split, split_reindexer = nlmod.layers.split_layers_ds(
    regis, split_dict, return_reindexer=True
)

View the resulting Dataset:

In [ ]:
regis_split

The reindexer dictionary links the new layers to the old layers. This can be convenient
for copying data from the original layers to the new sub-layers.

In [ ]:
# key = new layer index
# value = original layer index: repeats where layer was split
split_reindexer

Plot the cross-section of the original and the new layer model.

In [ ]:
compare_layer_models(regis, line, colors, ds2=regis_split, title2="Split layers")

## Combine layers

Example how to combine model layers. First find the indices of the layers to combine.

In [ ]:
combine_layers = [
    tuple(np.argwhere(regis.layer.str.startswith("URz").data).squeeze().tolist()),
    tuple(np.argwhere(regis.layer.isin(["PZWAz2", "PZWAz3"]).data).squeeze().tolist()),
]
combine_layers

Combine layers using the `combine_layers_ds()` function and passing the layer dataset and the list of layer numbers to combine.

In [ ]:
regis_combined = nlmod.layers.combine_layers_ds(regis, combine_layers, kD=None, c=None)

Take a look a the resulting dataset

In [ ]:
regis_combined

Plot the new and the old cross-section. Use the layer code and color from the first layer name for the combined layer

In [ ]:
compare_layer_models(regis, line, colors, ds2=regis_combined, title2="Combined layers")

## Set new model top

The `nlmod.layers.set_model_top` changes the top of the model. When the new top is
lower than the old top, the new top is burned in the layer model, lowering the top of
the top layer(s). Top layers can become incactive, when the thickness is reduced to 0.
When the new top is higher than the old top, the thickness of the most upper active
layer (not necessarily the first) is increased. This method can be used to change the
model top to a digital terrain model with a higher accuracy.

First transform the regis-date to a model Dataset, as the next methods need a model
Dataset.

In [ ]:
ds = nlmod.to_model_ds(regis)

In [ ]:
ds_new = nlmod.layers.set_model_top(ds.copy(deep=True), 5.0)
compare_layer_models(ds, line, colors, ds2=ds_new, title2="New top")

## Set layer top
`nlmod.layers.set_layer_top` sets the layer top to a specified value or array.

This method only changes the shape of the layer, and does not check if all hydrological properties are defined for cells that had a thickness of 0 before.

In [ ]:
ds_new = nlmod.layers.set_layer_top(ds.copy(deep=True), "WAk1", -40.0)
compare_layer_models(ds, line, colors, ds2=ds_new, title2="Modified")

## Set layer bottom
`nlmod.layers.set_layer_botm` sets the layer botm to a specified value or array.

This method only changes the shape of the layer, and does not check if all hydrological properties are defined for cells that had a thickness of 0 before.

In [ ]:
# set the botm of 'WAk1' to -70 m NAP
ds_new = nlmod.layers.set_layer_botm(ds.copy(deep=True), "WAk1", -70.0)
compare_layer_models(ds, line, colors, ds2=ds_new, title2="Modified")

## Set layer thickness
`nlmod.layers.set_layer_thickness` sets the thickness of a layer to a specified value or array.  With a parameter called 'change' you can specify in which direction the layer is changed. The only supported option for now is 'botm', which changes the layer botm. 

This method only changes the shape of the layer, and does not check if all hydrological properties are defined for cells that had a thickness of 0 before.

In [ ]:
# set the thickness of 'WAk1' to 20 m NAP
ds_new = nlmod.layers.set_layer_thickness(ds.copy(deep=True), "WAk1", 20)
compare_layer_models(ds, line, colors, ds2=ds_new, title2="Modified")

## Set minimum layer thickness
`nlmod.layers.set_minimum layer_thickness` increases the thickness of a layer if the thickness is less than a specified value.  With a parameter called 'change' you can specify in which direction the layer is changed. The only supported option for now is 'botm', which changes the layer botm. 

This method only changes the shape of the layer, and does not check if all hydrological properties are defined for cells that had a thickness of 0 before.

In [ ]:
# set the minimum thickness of 'PZWAz2' to 20 m
ds_new = nlmod.layers.set_minimum_layer_thickness(ds.copy(deep=True), "PZWAz2", 20.0)
compare_layer_models(ds, line, colors, ds2=ds_new, title2="Modified")